In [3]:
import os
import gc
import time
import pickle
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import math
from collections import defaultdict

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [4]:
cache_path = './input/'
train_path = 'train_user.csv'
shop_path = 'train_shop.csv'
test_path = 'test.csv'
train_path = cache_path + train_path
test_path = cache_path + test_path
shop_path = cache_path + shop_path
wifi_shop_info = pd.DataFrame()
wifi_shop_connect_only = pd.DataFrame()
shop_loc = pd.DataFrame()
wifi_shop_count = pd.DataFrame()
flag = True

# 经纬度及时间特征

In [3]:
def get_time_hour(data):
    data['hourofday'] = pd.DatetimeIndex(data.time_stamp).hour
    data['dayofweek'] = pd.DatetimeIndex(data.time_stamp).day
    return data


# 相差的分钟数
def diff_of_minutes(time1, time2):
    d = {
        '5': 0,
        '6': 31,
    }
    try:
        days = (d[time1[6]] + int(time1[8:10])) - (
            d[time2[6]] + int(time2[8:10]))
        try:
            minutes1 = int(time1[11:13]) * 60 + int(time1[14:16])
        except:
            minutes1 = 0
        try:
            minutes2 = int(time2[11:13]) * 60 + int(time2[14:16])
        except:
            minutes2 = 0
        return (days * 1440 - minutes2 + minutes1)
    except:
        return np.nan


# 计算两点之间距离
def call_distance(lat1, lon1, lat2, lon2):
    dx = np.abs(lon1 - lon2)  # 经度差
    dy = np.abs(lat1 - lat2)  # 维度差
    b = (lat1 + lat2) / 2.0
    Lx = 6371004.0 * (dx / 57.2958) * np.cos(b / 57.2958)
    Ly = 6371004.0 * (dy / 57.2958)
    L = (Lx**2 + Ly**2)**0.5
    return L


def call_orientation(lat1, lon1, lat2, lon2):
    radLat1 = math.radians(lat1)
    radLon1 = math.radians(lon1)
    radLat2 = math.radians(lat2)
    radLon2 = math.radians(lon2)
    dLon = radLon2 - radLon1
    y = math.sin(dLon) * math.cos(radLat2)
    x = math.cos(radLat1) * math.sin(radLat2) - math.sin(radLat1) * math.cos(
        radLat2) * math.cos(dLon)
    brng = math.degrees(math.atan2(y, x))
    brng = (brng + 360) % 360
    return brng


def get_label(data):
    true = dict(zip(train['row_id'].values, train['shop_id']))
    print('label map finish')
    data['label'] = data['row_id'].map(true)
    data['label'] = (data['label'] == data['shop_id']).astype('int')
    return data


def make_wifi_shop_relation(train):
    fuck = train.values
    res1 = []
    res2 = []
    res3 = []
    for i in fuck:
        wifis = i[1].split(';')
        for j in wifis:
            wifi = j.split('|')
            res1.append(i[0])
            res2.append(wifi[0])
            res3.append(int(wifi[1]))
    t_data = pd.DataFrame()
    t_data['shop_id'] = res1
    t_data['wifi_id'] = res2
    t_data['power'] = res3
    return t_data


def make_wifi_rowid_relation(test):
    fuck = test.values
    res1 = []
    res2 = []
    res3 = []
    for i in fuck:
        wifis = i[1].split(';')
        for j in wifis:
            wifi = j.split('|')
            res1.append(i[0])
            res2.append(wifi[0])
            res3.append(int(wifi[1]))
    t_data = pd.DataFrame()
    t_data['row_id'] = res1
    t_data['wifi_id'] = res2
    t_data['power'] = res3
    return t_data


def wifi_shop_connect(train):
    fuck = train[['shop_id', 'wifi_infos']].values
    res1 = []
    res2 = []
    res3 = []
    for i in fuck:
        wifis = i[1].split(';')
        for j in wifis:
            wifi = j.split('|')
            res1.append(i[0])
            res2.append(wifi[0])
            if wifi[2] == 'true':
                res3.append(1)
            else:
                res3.append(0)
    t_data = pd.DataFrame()
    t_data['shop_id'] = res1
    t_data['wifi_id'] = res2
    t_data['connect'] = res3
    t_data = t_data[t_data.connect == 1]
    return t_data


def wifi_rowid_conenct(test):
    fuck = test[['row_id', 'wifi_infos']].values
    res1 = []
    res2 = []
    res3 = []
    for i in fuck:
        wifis = i[1].split(';')
        for j in wifis:
            wifi = j.split('|')
            res1.append(i[0])
            res2.append(wifi[0])
            if wifi[2] == 'true':
                res3.append(1)
            else:
                res3.append(0)
    t_data = pd.DataFrame()
    t_data['row_id'] = res1
    t_data['wifi_id'] = res2
    t_data['connect'] = res3
    t_data = t_data[t_data.connect == 1]
    return t_data

# 按WiFi构造负样本

In [22]:
train_feat.shape
train_feat.label.sum()
train_feat.head()

(3088310, 44)

278579

,row_id,shop_id,connectcount,user_id,time_stamp,longitude,latitude,wifi_infos,user_count,shop_hot,user_shop_count,category_id,longitude_s,latitude_s,price,mall_id,category,user_kind_count,wifi1,wifi2,wifi1_id,wifi1_power,wifi2_id,wifi2_power,ave_power1,power_dif1,ave_power2,power_dif2,w3,knn_values,connect1,connect2,connect_sum,hourofday,dayofweek,lon_real,lat_real,real_dis_df,real_ang_df,dis_df,ang_df,real_lat_dif,real_lon_dif,label
0,t1000,s_270092,1,168383,2017-08-30 16:50,114.435632,23.559487,b_8086987|-70|false;b_38106941|-71|false;b_412...,0.0,27,0.0,c_34,114.436631,23.559819,40,4459,34,0.0,"(b_52747133, -31)","(b_38106940, -52)",b_52747133,-31,b_38106940,-52,-42.750000,11.750000,-57.760000,5.760000,9,<map object at 0x000002062C6CF2B0>,1.0,0.0,1.0,16,30,114.435679,23.559549,5.919791,331.433110,112.128609,352.171965,0.000062,0.000047,1
1,t1000,s_3595309,0,168383,2017-08-30 16:50,114.435632,23.559487,b_8086987|-70|false;b_38106941|-71|false;b_412...,0.0,13,0.0,c_14,114.436320,23.558731,48,4459,14,0.0,"(b_52747133, -31)","(b_38106940, -52)",b_52747133,-31,b_38106940,-52,-77.000000,46.000000,-55.615385,3.615385,9,<map object at 0x000002062C6CF2B0>,0.0,0.0,0.0,16,30,114.435618,23.559805,14.718382,264.121899,84.034958,24.445121,0.000318,0.000014,0
2,t1000000,s_1027638,1,23097338,2017-08-30 12:20,120.354054,32.350553,b_54588167|-63|false;b_49790304|-67|false;b_55...,0.0,212,0.0,c_34,120.353872,32.350522,41,4923,34,0.0,"(b_57190679, -44)","(b_54588167, -63)",b_57190679,-44,b_54588167,-63,-69.081967,25.081967,-58.931507,4.068493,8,<map object at 0x000002062C6CF2B0>,1.0,0.0,1.0,12,30,120.353347,32.350544,78.636169,179.635234,20.312311,175.080434,0.000009,0.000707,0
3,t1000000,s_2391477,1,23097338,2017-08-30 12:20,120.354054,32.350553,b_54588167|-63|false;b_49790304|-67|false;b_55...,0.0,75,0.0,c_28,120.353749,32.350417,41,4923,28,0.0,"(b_57190679, -44)","(b_54588167, -63)",b_57190679,-44,b_54588167,-63,-71.000000,27.000000,-111.000000,48.000000,3,<map object at 0x000002062C6CF2B0>,0.0,0.0,0.0,12,30,120.353480,32.350505,63.864959,177.555538,34.764793,167.301583,0.000048,0.000574,0
4,t1000000,s_3340392,0,23097338,2017-08-30 12:20,120.354054,32.350553,b_54588167|-63|false;b_49790304|-67|false;b_55...,0.0,14,0.0,c_34,120.353754,32.350574,39,4923,34,0.0,"(b_57190679, -44)","(b_54588167, -63)",b_57190679,-44,b_54588167,-63,-71.500000,27.500000,-80.000000,17.000000,6,<map object at 0x000002062C6CF2B0>,0.0,0.0,0.0,12,30,120.353790,32.350527,29.429427,177.203781,33.379351,182.025914,0.000026,0.000264,0


In [29]:
len(list(train_feat.iloc[0].knn_values))

3072191

In [4]:
#构造wifishop对应信息
def get_wifi_shop_info(train):
    wifi_train = train[['shop_id', 'wifi_infos']].drop_duplicates()
    wifi_shop_info = make_wifi_shop_relation(wifi_train)
    wifi_shop_info = wifi_shop_info.groupby(
        ['shop_id', 'wifi_id'], as_index=False)['power'].agg({
            'ave_power':
            'mean'
        })
    return wifi_shop_info


def get_wifi_shop_count(train):
    wifi_train = train[['shop_id', 'wifi_infos']].drop_duplicates()
    wifi_shop_count = make_wifi_shop_relation(wifi_train)
    wifi_shop_count = wifi_shop_count.groupby(
        ['shop_id', 'wifi_id'], as_index=False)['power'].agg({
            'wifi inshopcount':
            'count'
        })
    wifi_shop_count.sort_values(['shop_id', 'wifi inshopcount'], inplace=True)
    wifi_shop_count = wifi_shop_count.groupby('shop_id').tail(50)
    return wifi_shop_count


# 通过wifi来构建负样本
def get_wifi_shop(train, test):
    result_path = cache_path + 'wifi_shop_%d.hdf' % (
        train.shape[0] * test.shape[0])
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        #唯一对应shop 和wifi  还可以通过能量去删选 这部分还没做
        #have connected
        wifi_rowid_connect = wifi_rowid_conenct(test)
        global wifi_shop_connect_only
        result1 = pd.merge(
            wifi_rowid_connect[['row_id', 'wifi_id']],
            wifi_shop_connect_only[['wifi_id', 'shop_id']],
            on=['wifi_id'])
        result1 = result1[['row_id', 'shop_id']].drop_duplicates()
        result1['canconnect'] = 1

        wifi_shop = wifi_shop_info[['shop_id', 'wifi_id',
                                    'ave_power']].drop_duplicates()
        # wifi_shop=wifi_shop[wifi_shop['ave_power']>=-80]
        # b=wifi_shop.sort_values('ave_power')
        #wifi_shop= b.groupby('wifi_id').tail(9)
        wifi_test = test[['row_id', 'wifi_infos']].drop_duplicates()
        wifi_rowid = make_wifi_rowid_relation(wifi_test)
        #for those not connected use the power
        wifi_rowid = wifi_rowid[~wifi_rowid['row_id'].isin(result1['row_id'])]
        #选取能量最大的三个看看有几个
        result3 = pd.merge(
            wifi_rowid[['row_id', 'wifi_id']],
            wifi_shop_connect_only[['wifi_id', 'shop_id']],
            on='wifi_id',
            how='left')
        result3 = result3[['row_id', 'shop_id']].drop_duplicates()
        result3['canconnect'] = 1
        #wifi_rowid=wifi_rowid[wifi_rowid['power']>=-80]
        a = wifi_rowid.sort_values('power')
        wifi_rowid = a.groupby('row_id').tail(2)
        result2 = pd.merge(
            wifi_rowid[['row_id', 'wifi_id']],
            wifi_shop,
            on='wifi_id',
            how='left')
        result2 = result2[['row_id', 'shop_id']].drop_duplicates()
        result2['canconnect'] = 0
        result = pd.concat([result1, result2, result3]).drop_duplicates()
        result = result[(~result['shop_id'].isnull())]
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result


def get_user_end_shop(train, test):
    result_path = cache_path + 'user_end_shop_%d.hdf' % (
        train.shape[0] * test.shape[0])
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        train_new = pd.merge(train, shop1, on='shop_id', how='left')
        user_shop = train_new[['shop_id', 'user_id',
                               'mall_id']].drop_duplicates()
        result = pd.merge(
            test[['user_id', 'mall_id', 'row_id']],
            user_shop,
            on=['user_id', 'mall_id'])
        result = result[['row_id', 'shop_id']].drop_duplicates()
        result['canconnect'] = 0
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result

# 提取用户特征

In [5]:
# 获取用户历史行为次数
def get_user_count(train, result):
    user_count = train.groupby(
        'user_id', as_index=False)['shop_id'].agg({
            'user_count': 'count'
        })
    result = pd.merge(result, user_count, on=['user_id'], how='left')
    return result


# hot of shop
def get_shop_hot(train, result):
    shop_hot = train.groupby(
        'shop_id', as_index=False)['row_id'].agg({
            'shop_hot': 'count'
        })
    result = pd.merge(result, shop_hot, on=['shop_id'], how='left')
    return result


# 获取用户到某商店的次数
def get_user_shop_count(train, result):
    user_shop_count = train.groupby(
        ['user_id', 'shop_id'], as_index=False)['user_id'].agg({
            'user_shop_count':
            'count'
        })
    result = pd.merge(
        result, user_shop_count, on=['user_id', 'shop_id'], how='left')
    return result


# 用户的平均消费能力 最大消费能力 最小消费

# 提取商店特征

In [6]:
#商店属性 商店所属类别，人均消费指数 所属商场号
def get_shop_info(result):
    result = pd.merge(result, shop, on=['shop_id'], how='left')
    result['category_id'] = result['category_id'].fillna(0)
    result['category'] = result['category_id'].map(
        lambda x: '0' if x == 0 else str(x)[2:]).astype('int')
    return result


#用户到某个类型的商店去过几次 是否去过这个类型的商店
def get_uesr_kind(train, result):
    user_kind_count = train.groupby(
        ['user_id', 'category_id'], as_index=False)['category_id'].agg({
            'user_kind_count':
            'count'
        })
    result = pd.merge(
        result, user_kind_count, on=['user_id', 'category_id'], how='left')
    return result

# 提取WiFi特征

In [8]:
wifi_shop_connect_only.drop_duplicates(['shop_id']).shape
wifi_shop_connect_only.shape

KeyError: 'shop_id'

In [7]:
#有没有连接这个wifi
#这个wifi的
#推荐的店铺和最强 第二强 第三强 第四强wifi的值 现实值 和差值
#强度排名
def f1(x):
    wifipower = {}
    wifi = []
    power = []
    wifis = x.split(';')
    for i in wifis:
        s = i.split('|')
        wifi.append(s[0])
        power.append(int(s[1]))
    wifipower = dict(zip(wifi, power))
    wifipower = sorted(wifipower.items(), key=lambda x: x[1], reverse=True)
    return wifipower[0]


def f2(x):
    wifipower = {}
    wifi = []
    power = []
    wifis = x.split(';')
    for i in wifis:
        s = i.split('|')
        wifi.append(s[0])
        power.append(int(s[1]))
    wifipower = dict(zip(wifi, power))
    wifipower = sorted(wifipower.items(), key=lambda x: x[1], reverse=True)
    if (len(wifipower) > 1):
        return wifipower[1]
    else:
        return 0


def get_wifi(result):
    result['wifi1'] = result['wifi_infos'].map(f1)
    result['wifi2'] = result['wifi_infos'].map(f2)
    result['wifi1_id'] = result['wifi1'].map(lambda x: x[0])
    result['wifi1_power'] = result['wifi1'].map(lambda x: x[1])
    result['wifi2_id'] = result['wifi2'].map(lambda x: 0 if x == 0 else x[0])
    result['wifi2_power'] = result['wifi2'].map(
        lambda x: 0 if x == 0 else x[1])
    wifi_shop_info.rename(columns={'wifi_id': 'wifi1_id'}, inplace=True)
    result = pd.merge(
        result, wifi_shop_info, on=['shop_id', 'wifi1_id'], how='left')
    result.rename(columns={'ave_power': 'ave_power1'}, inplace=True)
    result['ave_power1'] = result['ave_power1'].fillna(
        result['ave_power1'].min())
    result['power_dif1'] = abs(
        (result['ave_power1'] - result['wifi1_power']).astype(np.float))
    wifi_shop_info.rename(columns={'wifi1_id': 'wifi2_id'}, inplace=True)
    result = pd.merge(
        result, wifi_shop_info, on=['shop_id', 'wifi2_id'], how='left')
    wifi_shop_info.rename(columns={'wifi2_id': 'wifi_id'}, inplace=True)
    result.rename(columns={'ave_power': 'ave_power2'}, inplace=True)
    result['ave_power2'] = result['ave_power2'].fillna(
        result['ave_power2'].min())
    result['power_dif2'] = abs(
        (result['ave_power2'] - result['wifi2_power']).astype(np.float))
    return result


def wifi_name(x):
    wifi = []
    wifis = x.split(';')
    for i in wifis:
        s = i.split('|')
        wifi.append(s[0])
    return wifi


def fuck(x):
    x = list(x.split(':'))
    return x


def get_same_relation(result):
    result['w1'] = result['wifi_infos'].map(wifi_name)
    t2 = wifi_shop_count.groupby('shop_id')['wifi_id'].agg(
        lambda x: ':'.join(x)).reset_index()
    t2['w2'] = t2['wifi_id'].map(fuck)
    del t2['wifi_id']
    result = pd.merge(result, t2, on=['shop_id'], how='left')
    a = list(result['w1'])
    b = list(result['w2'])
    c = []
    for i in range(len(a)):
        try:
            d = len(set(a[i]) & set(b[i]))
            c.append(d)
        except:
            c.append(0)
    result['w3'] = c
    del result['w1']
    del result['w2']
    return result


def get_wifi_power(x):
    wifipower = {}
    wifi = []
    power = []
    wifis = x.split(';')
    for i in wifis:
        s = i.split('|')
        wifi.append(s[0])
        power.append(int(s[1]))
    wifipower = dict(zip(wifi, power))
    return wifipower


def get_shop_wifi_dic(x):
    wifipower = {}
    wifi = []
    power = []
    wifis = x.split('|')
    for i in wifis:
        s = i.split(':')
        wifi.append(s[0])
        power.append(float(s[1]))
    wifipower = dict(zip(wifi, power))
    return wifipower


def get_knn(x, y):
    sum = 0
    for each in x:
        try:
            if each in y:
                sum += (x[each] - y[each])**2  #历史信息里存在这个信号的话，计算
            else:
                sum += (x[each] - (-90))**2
        except:
            sum = 999
    return sum**0.5 * 1.0 / len(x)


def ger_power_var(result):
    result['w4'] = result['wifi_infos'].map(get_wifi_power)
    wifi_shop_info['w'] = wifi_shop_info['wifi_id'] + ':' + wifi_shop_info['ave_power'].astype('str')
    t1 = wifi_shop_info[['shop_id', 'w']]
    t2 = t1.groupby('shop_id')['w'].agg(lambda x: '|'.join(x)).reset_index()
    t2['w'] = t2['w'].map(get_shop_wifi_dic)
    result = pd.merge(result, t2, on=['shop_id'], how='left')
    result['knn_values'] = map(lambda x, y: get_knn(x, y), result['w4'],
                               result['w'])
    del result['w4']
    del result['w']
    return result


#shop_id和当前信号最强的wifi以及第二强的wifi有没有互联记录
def if_wifi_shop_connected(result):
    global wifi_shop_connect_only
    wifi_shop_connect_only.rename(
        columns={'wifi_id': 'wifi1_id',
                 'connect': 'connect1'}, inplace=True)
    result = pd.merge(
        result, wifi_shop_connect_only, on=['wifi1_id', 'shop_id'], how='left')
    wifi_shop_connect_only.rename(
        columns={'wifi1_id': 'wifi2_id',
                 'connect1': 'connect2'}, inplace=True)
    result = pd.merge(
        result, wifi_shop_connect_only, on=['wifi2_id', 'shop_id'], how='left')
    wifi_shop_connect_only.rename(
        columns={'wifi2_id': 'wifi_id',
                 'connect2': 'connect'}, inplace=True)
    result['connect1'] = result['connect1'].fillna(0)
    result['connect2'] = result['connect2'].fillna(0)
    result['connect_sum'] = result['connect1'] + result['connect2']
    return result

# 空间特征

In [8]:
#找一下距离差
#加上商店历史上的平均距离，商店位置表中的距离不准
def get_real_shop_loc(result):
    global shop_loc
    shop_loc = shop_loc.groupby(
        'shop_id', as_index=False).agg({
            'longitude': 'mean',
            'latitude': 'mean'
        })
    shop_loc.rename(
        columns={'longitude': 'lon_real',
                 'latitude': 'lat_real'},
        inplace=True)
    result = pd.merge(result, shop_loc, on='shop_id', how='left')
    shop_loc.rename(
        columns={'lon_real': 'longitude',
                 'lat_real': 'latitude'},
        inplace=True)
    return result


def distance_real_dif(result):
    loc1 = list(zip(result.longitude, result.latitude))
    loc2 = list(zip(result.lon_real, result.lat_real))
    sum = list(zip(loc1, loc2))
    dis_df1 = []
    for i in sum:
        lat1, lon1 = i[0]
        lat2, lon2 = i[1]
        d = call_distance(lat1, lon1, lat2, lon2)
        dis_df1.append(d)
    result['real_dis_df'] = dis_df1
    return result


def angle_real_dif(result):
    loc1 = list(zip(result.longitude, result.latitude))
    loc2 = list(zip(result.lon_real, result.lat_real))
    sum = list(zip(loc1, loc2))
    ang_df1 = []
    for i in sum:
        lat1, lon1 = i[0]
        lat2, lon2 = i[1]
        d = call_orientation(lat1, lon1, lat2, lon2)
        ang_df1.append(d)
    result['real_ang_df'] = ang_df1
    return result


def distance_dif(result):
    loc1 = list(zip(result.longitude, result.latitude))
    loc2 = list(zip(result.longitude_s, result.latitude_s))
    sum = list(zip(loc1, loc2))
    dis_df1 = []
    for i in sum:
        lat1, lon1 = i[0]
        lat2, lon2 = i[1]
        d = call_distance(lat1, lon1, lat2, lon2)
        dis_df1.append(d)
    result['dis_df'] = dis_df1
    return result


#找一下角度差
def angle_dif(result):
    loc1 = list(zip(result.longitude, result.latitude))
    loc2 = list(zip(result.longitude_s, result.latitude_s))
    sum = list(zip(loc1, loc2))
    ang_df1 = []
    for i in sum:
        lat1, lon1 = i[0]
        lat2, lon2 = i[1]
        d = call_orientation(lat1, lon1, lat2, lon2)
        ang_df1.append(d)
    result['ang_df'] = ang_df1
    return result


#行为发生时间
def get_time(result):
    result = get_time_hour(result)
    return result

# 构造样本及制作训练集

In [9]:
def get_sample(train, test):
    result_path = cache_path + 'sample_%d.hdf' % (
        train.shape[0] * test.shape[0])
    if os.path.exists(result_path) & flag:
        result = pd.read_hdf(result_path, 'w')
    else:
        result1 = get_wifi_shop(train, test)  # 根据wifi历史来添加样本
        result2 = get_user_end_shop(train, test)
        result = pd.concat([result1, result2]).drop_duplicates()
        result = result.groupby(
            ['row_id', 'shop_id'], as_index=False)['canconnect'].agg({
                'connectcount':
                'sum'
            })
        test_temp = test.copy()
        result = pd.merge(result, test_temp, on='row_id', how='left')
        del result['mall_id']
        result.to_hdf(result_path, 'w', complib='blosc', complevel=5)
    return result


def make_train_set(train, test):
    global wifi_shop_count
    global wifi_shop_info
    global wifi_shop_connect_only
    wifi_shop_connect1 = wifi_shop_connect(train)
    wifi_shop_connect_only = wifi_shop_connect1.drop_duplicates()
    print('make wifishop info')
    wifi_shop_info = get_wifi_shop_info(train)
    wifi_shop_count = get_wifi_shop_count(train)
    print('make sample...')
    result = get_sample(train, test)  # 构造备选样本
    del test['mall_id']
    train = pd.merge(train, shop, on=['shop_id'], how='left')

    print('make character...')

    result = get_user_count(train, result)  # 获取用户历史行为次数
    result = get_shop_hot(train, result)
    result = get_user_shop_count(train, result)  # 获取用户去过这个商店几次以及是否去过
    result = get_shop_info(result)
    result = get_uesr_kind(train, result)
    result = get_wifi(result)
    result = get_same_relation(result)
    result = ger_power_var(result)
    result = if_wifi_shop_connected(result)
    result = get_time(result)
    result = get_real_shop_loc(result)
    result = distance_real_dif(result)
    result = angle_real_dif(result)
    result = distance_dif(result)
    result = angle_dif(result)
    result['user_id'] = result.user_id.map(lambda x: int(str(x)[2:]))
    result['mall_id'] = result.mall_id.map(lambda x: int(str(x)[2:]))
    result['real_lat_dif'] = abs(result['lat_real'] - result['latitude'])
    result['real_lon_dif'] = abs(result['lon_real'] - result['longitude'])
    result.fillna(0, inplace=True)
    print('result.columns:\n{}'.format(result.columns))
    return result

# 训练提交

In [20]:
predictors = [
    'knn_values', 'longitude_s', 'latitude_s', 'real_lon_dif', 'real_lat_dif',
    'w3', 'lon_real', 'lat_real', 'real_ang_df', 'real_dis_df', 'connectcount',
    'connect_sum', 'ang_df', 'dis_df', 'mall_id', 'user_id', 'dayofweek',
    'hourofday', 'connect1', 'connect2', 'power_dif2', 'ave_power2',
    'power_dif1', 'ave_power1', 'wifi2_power', 'wifi1_power',
    'user_kind_count', 'category', 'price', 'user_shop_count', 'shop_hot',
    'user_count'
]

In [21]:
x_train = train_feat[predictors]
x_train.shape
x_train.iloc[0]

(3088310, 32)

knn_values         <map object at 0x000002062C6CF2B0>
longitude_s                                   114.437
latitude_s                                    23.5598
real_lon_dif                              4.67568e-05
real_lat_dif                              6.15405e-05
w3                                                  9
lon_real                                      114.436
lat_real                                      23.5595
real_ang_df                                   331.433
real_dis_df                                   5.91979
connectcount                                        1
connect_sum                                         1
ang_df                                        352.172
dis_df                                        112.129
mall_id                                          4459
user_id                                        168383
dayofweek                                          30
hourofday                                          16
connect1                    

In [18]:
%%time
if __name__ == "__main__":
    t0 = time.time()
    train = pd.read_csv(train_path)
    test = pd.read_csv(test_path)
    shop = pd.read_csv(shop_path)
    shop_loc = train[['shop_id', 'longitude', 'latitude']]
    shop.rename(columns={'longitude': 'longitude_s'}, inplace=True)
    shop.rename(columns={'latitude': 'latitude_s'}, inplace=True)
    train['row_id'] = range(train.shape[0])
    train['row_id'] = 't' + train['row_id'].astype('str')
    shop1 = shop[['shop_id', 'mall_id']]
    train1 = train[(train['time_stamp'] < '2017-08-25 00:00:00')]
    train2 = train[(train['time_stamp'] >= '2017-08-25 00:00:00')]
    train2 = pd.merge(train2, shop1, on=['shop_id'], how='left')
    del train2['shop_id']
    print('make train_feat')

    train_feat = make_train_set(train1, train2)
    print('add label')
    train_feat = get_label(train_feat)
    print('finish add_label')
    print('pos/all=%f' % (len(train_feat[train_feat['label'] == 1]) * 1.0 /
                          (train2.shape[0])))

    print('make test_feat')

    test_feat = make_train_set(train, test)

#     import xgboost as xgb
#     predictors = ['knn_values','longitude_s', 'latitude_s','real_lon_dif','real_lat_dif','w3','lon_real','lat_real','real_ang_df','real_dis_df','connectcount','connect_sum','ang_df','dis_df','mall_id','user_id','dayofweek','hourofday','connect1','connect2','power_dif2','ave_power2','power_dif1',
#     'ave_power1','wifi2_power','wifi1_power','user_kind_count','category','price','user_shop_count',
#    'shop_hot','user_count']
#     params = {
#         'objective': 'binary:logistic',
#         'eta': 0.05,
#         'colsample_bytree': 0.886,
#         'min_child_weight': 1.1,
#         'max_depth': 7,
#         'subsample': 0.886,
#         'gamma': 0.1,
#         'lambda':10,
#         'verbose_eval': True,
#         'eval_metric': 'auc',
#         'scale_pos_weight': 8,
#         'seed': 201703,
#         'missing':-1
#    }
#     xgbtrain = xgb.DMatrix(train_feat[predictors], train_feat['label'])
#     xgbtest = xgb.DMatrix(test_feat[predictors])
#     print 'start training'
#     model = xgb.train(params, xgbtrain, num_boost_round=1000)
#     gc.collect()
#     print 'start predicting'
#     test_feat.loc[:,'pred'] = model.predict(xgbtest)
#     test_feat = test_feat[['row_id','shop_id','pred']].drop_duplicates()
#     gc.collect()
#     result=test_feat.groupby('row_id',as_index=False)['pred'].agg({'pred':'max'})
#     result=pd.merge(result,test_feat,on=['row_id','pred'],how='left')
#     test = pd.read_csv(test_path)
#     result = pd.merge(test[['row_id']],result[['row_id','shop_id']],on='row_id',how='left')
#     result.fillna('0',inplace=True)
#     result.to_csv('result.csv',index=False)
#     print('一共用时{}秒'.format(time.time()-t0))
#     xgb.plot_importance(model)

make train_feat
make wifishop info
make sample...
make character...
result.columns:
Index(['row_id', 'shop_id', 'connectcount', 'user_id', 'time_stamp',
       'longitude', 'latitude', 'wifi_infos', 'user_count', 'shop_hot',
       'user_shop_count', 'category_id', 'longitude_s', 'latitude_s', 'price',
       'mall_id', 'category', 'user_kind_count', 'wifi1', 'wifi2', 'wifi1_id',
       'wifi1_power', 'wifi2_id', 'wifi2_power', 'ave_power1', 'power_dif1',
       'ave_power2', 'power_dif2', 'w3', 'knn_values', 'connect1', 'connect2',
       'connect_sum', 'hourofday', 'dayofweek', 'lon_real', 'lat_real',
       'real_dis_df', 'real_ang_df', 'dis_df', 'ang_df', 'real_lat_dif',
       'real_lon_dif'],
      dtype='object')
add label
label map finish
finish add_label
pos/all=0.965605
make test_feat
make wifishop info
make sample...
make character...
result.columns:
Index(['row_id', 'shop_id', 'connectcount', 'user_id', 'time_stamp',
       'longitude', 'latitude', 'wifi_infos', 'user_count